In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
csvfile = "Buoy Stations - Sheet1.csv" 

In [10]:
buoy_df_raw = pd.read_csv(csvfile)
buoy_df = buoy_df_raw.dropna()
buoy_df.head()

,Buoy,Lat,Long
0,42002,26.055,-93.646
1,42003,25.925,-85.615
2,khhv,26.939,-94.689
3,kemk,27.819,-94.323
4,42019,27.910,-95.345


In [21]:
lat_val = []
lng_val = []
for index,row in buoy_df.iterrows():
    lat_val.append(row["Lat"])
    lng_val.append(row["Long"])
    
print(lat_val)
print(lng_val)

[26.055, 25.925, 26.939, 27.819, 27.91, 27.354, 27.625, 26.968, 25.9, 29.232, 28.982, 28.351, 28.842, 29.635, 29.087, 27.774, 28.314, 22.124, 19.82, 27.463, 23.822, 23.37, 30.06, 28.787, 28.501, 27.505, 27.345, 28.501, 27.59, 25.701, 25.171, 14.394, 26.01, 27.173, 15.287, 16.433, 17.591, 17.695, 18.094, 17.869, 21.03]
[-93.646, -85.615, -94.689, -94.323, -95.345, -94.625, -90.441, -96.693, -89.287, -94.413, -94.899, -96.006, -94.242, -93.642, -92.506, -96.972, -95.62, -93.941, -84.945, -71.466, -68.384, -67.517, -87.548, -86.007, -84.508, -83.741, -84.275, -84.508, -82.931, -83.65, -83.475, -74.816, -83.086, -82.924, -67.477, -63.331, -61.821, -64.754, -65.471, -66.532, -64.79]


In [22]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [24]:
hour_temp =[]
hour_pressure =[]
hour_wind_speed =[]
hour_wind_deg =[]
for lat in lat_val[:5]:
    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat_val}&lon={lng_val}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        for i in range(len(hist_json["hourly"])):
            hour_temp.append(hist_json["hourly"][i]["temp"])
            hour_pressure.append(hist_json["hourly"][i]["pressure"])
            hour_wind_speed.append(hist_json["hourly"][i]["wind_speed"])
            hour_wind_deg.append(hist_json["hourly"][i]["wind_deg"])
        print(historical_url)
        print(hour_temp)
        print((hour_pressure))
        print((hour_wind_speed))
        print(len(hist_json["hourly"]))
        print((hour_wind_deg))
        
        
        




KeyError: 'hourly'

In [2]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [5]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [6]:
df_city.head(20)

,City,lat,lng
0,baytown,29.717449,-94.864024
1,stafford,29.489561,-95.559294
2,katy,29.712127,-96.111685
3,nacogdoches,31.462704,-94.444544
4,san angelo,31.323938,-100.195455
5,fredericksburg,30.036100,-98.846262
6,hondo,29.617627,-99.186429
7,robinson,31.334930,-96.486096
8,brownwood,31.742301,-98.733047
9,robinson,31.236295,-96.799196


In [7]:
#check length of cities
len(list(df_city["City"].unique()))

112

In [8]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

target_cities

['baytown',
 'stafford',
 'katy',
 'nacogdoches',
 'san+angelo',
 'fredericksburg',
 'hondo',
 'robinson',
 'brownwood',
 'el+campo',
 'kerrville',
 'copperas+cove',
 'lakeway',
 'wharton',
 'uvalde',
 'lufkin',
 'huntsville',
 'atascocita',
 'vidor',
 'beeville',
 'pleasanton',
 'natchitoches',
 'bryan',
 'belton',
 'victoria',
 'college+station',
 'liberty',
 'leander',
 'orange',
 'lockhart',
 'palestine',
 'galveston',
 'brenham',
 'taylor',
 'sulphur',
 'seguin',
 'lumberton',
 'port+arthur',
 'la+marque',
 'live+oak',
 'freeport',
 'san+antonio',
 'canyon+lake',
 'aldine',
 'pasadena',
 'angleton',
 'austin',
 'georgetown',
 'cedar+park',
 'san+marcos',
 'gatesville',
 'converse',
 'bay+city',
 'nederland',
 'del+rio',
 'bellmead',
 'sugar+land',
 'bridge+city',
 'west+university+place',
 'beaumont',
 'port+lavaca',
 'harker+heights',
 'temple',
 'leon+valley',
 'kirby',
 'jacinto+city',
 'schertz',
 'tomball',
 'pflugerville',
 'woodway',
 'conroe',
 'kyle',
 'eagle+pass',
 'lak

In [10]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

 base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
temp=[]
wind_speed=[]
pressure=[]
city_name=[]

for city in target_cities[:4]:
    query_url=f"{base_current_url}appid={weather_api_key}&q={city},US&units=imperial"
    req = requests.get(query_url)
    response1=req.json()
    latitude=response1["coord"]["lat"]
    longitude=response1["coord"]["lon"]

    for x in unix_time():
        historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
        hist_json = requests.get(historical_url).json()
        temp.append(hist_json["current"]["temp"])
        pressure.append(hist_json["current"]["pressure"])
        wind_speed.append(hist_json["current"]["wind_speed"])
        city_name.append(response1['name'])
        print(historical_url)

In [11]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    temp=[]
    wind_speed=[]
    pressure=[]
    hour_temp =[]
    hour_pressure =[]
    hour_wind_speed =[]
    city_name=[]
    country_code=[]
    for city in target_cities[:5]:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]
#             print(response1)

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                for i in range(len(hist_json["hourly"][0])):
                    hour_temp.append(hist_json["hourly"][0]["temp"])
                    hour_pressure.append(hist_json["hourly"][0]["pressure"])
                    hour_wind_speed.append(hist_json["hourly"][0]["wind_speed"])
                    #city_name.append(response1['name'])
                    #country_code.append(response1["sys"]["country"])
                
    d=dict()
    #d["City Name"] = city_name
    #d["Country"] = country_code
    d["Temperature"] = hour_temp
    d["Wind_Speed"] = hour_wind_speed
    d["Pressure"] = hour_pressure    
    return(d)
        #print ("--"*30)

In [12]:
unix_time()

[1630558800, 1630472400, 1630386000, 1630299600, 1630213200]

In [21]:

historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=26.055&lon=-93.646&dt=1630558800&appid={weather_api_key}&units=imperial"
#historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=31.369003&lon=-97.767841&dt=1630461738&appid={weather_api_key}&units=imperial"
hist_json = requests.get(historical_url).json()

hour_temp =[]
hour_pressure =[]
hour_wind_speed =[]
hour_wind_deg =[]
for i in range(len(hist_json["hourly"])):
    hour_temp.append(hist_json["hourly"][i]["temp"])
    hour_pressure.append(hist_json["hourly"][i]["pressure"])
    hour_wind_speed.append(hist_json["hourly"][i]["wind_speed"])
    hour_wind_deg.append(hist_json["hourly"][i]["wind_deg"])
print(historical_url)
print(hour_temp)
print((hour_pressure))
print((hour_wind_speed))
print(len(hist_json["hourly"]))
print((hour_wind_deg))

http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=26.055&lon=-93.646&dt=1630558800&appid=15a347531c589488d7fec7ef38ad8fac&units=imperial
[86.2, 86.22, 86.05, 85.89, 85.62, 85.39, 85.28, 85.14, 85.28, 85.59, 85.66, 85.17, 85.17, 85.73, 85.69, 86.07, 86.25, 86.2, 86.05, 86.04, 85.95, 86.02, 86, 85.98]
[1010, 1010, 1011, 1011, 1011, 1011, 1011, 1011, 1010, 1010, 1011, 1011, 1012, 1012, 1012, 1013, 1013, 1013, 1013, 1013, 1012, 1012, 1012, 1012]
[10.38, 12.08, 14.23, 15.28, 14.92, 14.43, 14.79, 15.28, 14.5, 14.12, 14.81, 16.06, 15.3, 15.32, 15.66, 14.43, 14.36, 13.98, 12.71, 12.3, 12.06, 10.71, 9.66, 8.88]
24
[144, 139, 143, 143, 143, 141, 145, 150, 152, 153, 155, 154, 160, 167, 171, 175, 177, 178, 175, 171, 166, 161, 159, 153]


In [16]:
#print(len(hist_json["hourly"]))

In [72]:
import json
print(json.dumps(hist_json, indent=4))

{
    "lat": 31.2071,
    "lon": -98.3381,
    "timezone": "America/Chicago",
    "timezone_offset": -18000,
    "current": {
        "dt": 1630461738,
        "sunrise": 1630411748,
        "sunset": 1630457906,
        "temp": 85.28,
        "feels_like": 90.09,
        "pressure": 1008,
        "humidity": 61,
        "dew_point": 70.3,
        "uvi": 0,
        "clouds": 40,
        "visibility": 10000,
        "wind_speed": 1.99,
        "wind_deg": 141,
        "wind_gust": 7,
        "weather": [
            {
                "id": 802,
                "main": "Clouds",
                "description": "scattered clouds",
                "icon": "03n"
            }
        ]
    },
    "hourly": [
        {
            "dt": 1630454400,
            "temp": 91.72,
            "feels_like": 96.94,
            "pressure": 1007,
            "humidity": 48,
            "dew_point": 69.24,
            "uvi": 0.31,
            "clouds": 40,
            "visibility": 10000,
            "w

In [14]:
df2 =pd.DataFrame(find_location(target_cities))
df2

,Temperature,Wind_Speed,Pressure
0,86.99,1.01,1008
1,86.99,1.01,1008
2,86.99,1.01,1008
3,86.99,1.01,1008
4,86.99,1.01,1008
...,...,...,...
325,84.49,17.27,1011
326,84.49,17.27,1011
327,84.49,17.27,1011
328,84.49,17.27,1011


In [20]:
df[df["Country"]=="US"]

,City Name,Country,Temperature,Wind_Speed,Pressure
0,Leander,US,78.82,1.99,1005
1,Leander,US,79.05,1.99,1008
2,Leander,US,74.95,1.01,1010
3,Leander,US,77.36,1.99,1010
4,Leander,US,78.57,0.00,1010
5,Port Arthur,US,78.60,4.61,1008
6,Port Arthur,US,78.60,10.36,1009
7,Port Arthur,US,76.21,5.75,1014
8,Port Arthur,US,77.77,4.61,1014
9,Port Arthur,US,79.34,11.50,1014
